In [ ]:
import cv2
import pandas as pd
from pdf2image import convert_from_path
from mmdet.apis import init_detector, inference_detector
import camelot


In [ ]:
filename = 'keppel-corporation-limited-annual-report-2018.pdf'
pages = convert_from_path(filename, 500, first_page=69, last_page=69)
page = pages[0]  # Since it’s a single page
image_path = 'page69.jpg'
page.save(image_path, 'JPEG')


In [ ]:
config_file = 'path/to/cascade_mask_rcnn_hrnetv2p_w32_20e.py'
checkpoint_file = 'path/to/checkpoint_file.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')


In [ ]:
result = inference_detector(model, image_path)

In [ ]:
tables = []
for bbox in result:
    x1, y1, x2, y2 = map(int, bbox)
    tables.append((x1, y1, x2, y2))


In [ ]:
dfs = []
for table in tables:
    x1, y1, x2, y2 = table
    # Camelot uses a different coordinate system, hence the transformations
    tables = camelot.read_pdf(filename, flavor='stream', table_areas=[f"{x1},{y1},{x2},{y2}"], pages="69")
    for table in tables:
        dfs.append(table.df)


In [ ]:
with pd.ExcelWriter('extracted_tables.xlsx') as writer:
    for i, df in enumerate(dfs):
        df.to_excel(writer, sheet_name=f'Table_{i}', index=False)